In [13]:
# @title Environment Setup
import os
import sys
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']
    print("MPLBACKEND environment variable cleared.")

# 2. Clone the repository
!rm -rf FontDiffusion
!git clone https://github.com/dzungphieuluuky/FontDiffusion.git

Cloning into 'FontDiffusion'...
remote: Enumerating objects: 20623, done.
remote: Counting objects: 100% (197/197), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 20623 (delta 126), reused 116 (delta 54), pack-reused 20426 (from 2)
Receiving objects: 100% (20623/20623), 278.02 MiB | 38.29 MiB/s, done.
Resolving deltas: 100% (972/972), done.
Updating files: 100% (141/141), done.


In [2]:
import os
import sys
from IPython import get_ipython
from typing import Optional

def configure_environment_paths():
    try:
        if "google.colab" in str(get_ipython()):
            print("✅ Environment: Google Colab")
            base_data_path = "/content/"
            base_output_path = "/content/"
            environment_name = "colab"
        elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
            print("✅ Environment: Kaggle")
            base_data_path = "/kaggle/input/"
            base_output_path = "/kaggle/working/"
            environment_name = "kaggle"
        else:
            print("⚠️ Environment: Local/Unknown")
            base_data_path = "./data/"
            base_output_path = "./output/"
            environment_name = "local"
    except NameError:
        print("⚠️ Non-interactive session. Using local paths.")
        base_data_path = "./data/"
        base_output_path = "./output/"
        environment_name = "local"
    os.makedirs(base_output_path, exist_ok=True)
    print(f"📂 Data Path: {base_data_path}")
    print(f"📦 Output Path: {base_output_path}")
    return base_data_path, base_output_path, environment_name

def load_secret(key_name: str) -> Optional[str]:
    env = ENV_NAME
    secret_value = None
    print(f"Attempting to load secret '{key_name}' from '{env}' environment...")
    try:
        if env == "colab":
            from google.colab import userdata
            secret_value = userdata.get(key_name)
        elif env == "kaggle":
            from kaggle_secrets import UserSecretsClient
            user_secrets = UserSecretsClient()
            secret_value = user_secrets.get_secret(key_name)
        else:
            secret_value = os.getenv(key_name)
        if not secret_value:
            print(f"⚠️ Secret '{key_name}' not found in the {env} environment.")
            return None
        print(f"✅ Successfully loaded secret '{key_name}'.")
        return secret_value
    except Exception as e:
        print(f"❌ An error occurred while loading secret '{key_name}': {e}")
        return None

def print_system_info():
    print("\n🔧 System Information")
    print(f"Python version: {sys.version.split()[0]}")
    try:
        import torch
        print(f"PyTorch version: {torch.__version__}")
        if torch.cuda.is_available():
            print(f"CUDA version: {torch.version.cuda}")
            print(f"GPU count: {torch.cuda.device_count()}")
            for i in range(torch.cuda.device_count()):
                print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        else:
            print("CUDA not available")
    except ImportError:
        print("PyTorch not installed")
    finally:
      !nvidia-smi

INPUT_PATH, OUTPUT_PATH, ENV_NAME = configure_environment_paths()
is_kaggle = ("kaggle" in ENV_NAME)
is_colab = not is_kaggle
print_system_info()

os.environ["WANDB_API_KEY"] = wandb_key = load_secret("WANDB_API_KEY")
os.environ["HF_TOKEN"] = HF_TOKEN = load_secret('HF_TOKEN')

# Now, these libraries will log in automatically
import wandb
import huggingface_hub

wandb.login() 
huggingface_hub.login(token=os.environ["HF_TOKEN"]) 

✅ Environment: Kaggle
📂 Data Path: /kaggle/input/
📦 Output Path: /kaggle/working/

🔧 System Information
Python version: 3.11.13
PyTorch version: 2.6.0+cu124
CUDA version: 12.4
GPU count: 2
  GPU 0: Tesla T4
  GPU 1: Tesla T4
Fri Jan  2 05:14:35 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A  

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [3]:
!uv pip install --upgrade pip
# 3. Install PyTorch 1.13
%cd {OUTPUT_PATH}
# Force reinstall torch 1.13 to match the model's training environment
# !uv pip uninstall torch torchvision
# !uv pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!uv pip install torch==2.9 torchvision
# 4. Install other dependencies

print("\n⬇️ Installing Dependencies (Manually fixed)...")
# Install xformers compatible with Torch 1.13
!uv pip install xformers==0.0.16 -q

# Install original dependencies
!uv pip install transformers==4.33.1 accelerate==0.23.0 diffusers==0.22.0
!uv pip install gradio==4.8.0 pyyaml pygame opencv-python info-nce-pytorch kornia
# -----------------------------------------------------------------
!uv pip install lpips scikit-image pytorch-fid
!sudo apt-get update && sudo apt-get install dos2unix
!uv pip install gdown tqdm
!uv pip install wandb
!uv pip install --upgrade pyarrow datasets
print("\n✅ Environment setup complete. You can now proceed to Block 2 (Inference).")

Using Python 3.11.13 environment at: /usr
Resolved 1 package in 265ms                                          
Prepared 1 package in 133ms                                              
Uninstalled 1 package in 252ms
Installed 1 package in 21ms                                 
 - pip==24.1.2
 + pip==25.3
/kaggle/working
Using Python 3.11.13 environment at: /usr
Resolved 40 packages in 556ms                                        
Prepared 19 packages in 38.65s                                           
Uninstalled 17 packages in 1.47s
Installed 19 packages in 285ms                              
 - nvidia-cublas-cu12==12.5.3.2
 + nvidia-cublas-cu12==12.8.4.1
 - nvidia-cuda-cupti-cu12==12.5.82
 + nvidia-cuda-cupti-cu12==12.8.90
 - nvidia-cuda-nvrtc-cu12==12.5.82
 + nvidia-cuda-nvrtc-cu12==12.8.93
 - nvidia-cuda-runtime-cu12==12.5.82
 + nvidia-cuda-runtime-cu12==12.8.90
 - nvidia-cudnn-cu12==9.3.0.75
 + nvidia-cudnn-cu12==9.10.2.21
 - nvidia-cufft-cu12==11.2.3.61
 + nvidia-cufft-cu12==11.

In [4]:
# KAGGLE CELL #1: Download checkpoint
if is_colab:
  !uv pip install --upgrade "huggingface-hub>=0.34.0,<1.0"
else:
  !uv pip install --upgrade "huggingface-hub==0.25.2" "protobuf<5.0.0" "numpy<2.0.0"
import os
import sys
from tqdm.auto import tqdm
from pathlib import Path
os.chdir(OUTPUT_PATH)
# Download from Hub
if not os.path.exists("ckpt") or not list(Path("ckpt").glob("*.safetensors")):
    print("📥 Downloading checkpoint from Hugging Face Hub...\n")
    from huggingface_hub import snapshot_download
    snapshot_download(
        repo_id="dzungpham/font-diffusion-weights",
        local_dir="ckpt",
        allow_patterns="*.safetensors",
        force_download=False
    )
    print("\n✅ Download complete!")
else:
    print("✅ Checkpoint already downloaded")
# Verify
print("\n📂 Files in ckpt/:")
for file in os.listdir("ckpt"):
    if file.endswith(".safetensors"):
        size = os.path.getsize(f"ckpt/{file}") / (1024**2)
        print(f"  ✓ {file} ({size:.2f} MB)")

Using Python 3.11.13 environment at: /usr
Resolved 14 packages in 149ms                                        
Prepared 4 packages in 423ms                                             
Uninstalled 4 packages in 42ms
Installed 4 packages in 24ms                                
 - fsspec==2025.10.0
 + fsspec==2025.12.0
 - huggingface-hub==1.2.3
 + huggingface-hub==0.25.2
 - numpy==2.4.0
 + numpy==1.26.4
 - protobuf==6.33.0
 + protobuf==4.25.8
📥 Downloading checkpoint from Hugging Face Hub...



Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

unet.safetensors:   0%|          | 0.00/315M [00:00<?, ?B/s]

content_encoder.safetensors:   0%|          | 0.00/4.76M [00:00<?, ?B/s]

style_encoder.safetensors:   0%|          | 0.00/82.4M [00:00<?, ?B/s]


✅ Download complete!

📂 Files in ckpt/:
  ✓ unet.safetensors (300.34 MB)
  ✓ content_encoder.safetensors (4.54 MB)
  ✓ style_encoder.safetensors (78.58 MB)


In [5]:
# @title Unzipping all archived files
import os
import glob
from zipfile import ZipFile

zip_file_paths = glob.glob(os.path.join(INPUT_PATH, '*.zip'))

if not zip_file_paths:
    print(f'No .zip files found in {INPUT_PATH}.')
else:
    for zip_file_path in zip_file_paths:
        if os.path.exists(zip_file_path):
            print(f'Unzipping {zip_file_path}...')
            !unzip -o {zip_file_path} -d ./
            print(f'Unzipping of {zip_file_path} complete.')
        else:
            print(f'Error: The file {zip_file_path} was not found (post-glob check).')

No .zip files found in /kaggle/input/.


In [6]:
import pandas as pd
import os
def convert_csv_to_chars_txt(input_csv_path: str, output_txt_path: str, column_name: str = 'word'):
    """
    Reads a CSV file, extracts text from a specified column, and writes each character
    to a new line in a plain text file.
    Args:
        input_csv_path (str): The full path to the input CSV file.
        output_txt_path (str): The full path for the output text file.
        column_name (str): The name of the column in the CSV file containing the text.
    """
    if not os.path.exists(input_csv_path):
        print(f"Error: Input CSV file not found at '{input_csv_path}'. Please ensure the file is uploaded.")
        return
    try:
        df = pd.read_csv(input_csv_path)
    except Exception as e:
        print(f"Error reading CSV file '{input_csv_path}': {e}")
        return
    if column_name not in df.columns:
        print(f"Error: Column '{column_name}' not found in the CSV file '{input_csv_path}'.")
        return
    all_characters = []
    for item in df[column_name].astype(str).dropna().tolist():
        for char in item:
            all_characters.append(char)
    os.makedirs(os.path.dirname(output_txt_path), exist_ok=True)
    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write("\n".join(all_characters))
    print(f"Successfully converted '{input_csv_path}' to '{output_txt_path}', with one character per line.")
print("\n--- Demonstrating function with a dummy CSV file ---")
dummy_csv_path = os.path.join(OUTPUT_PATH, "dummy_data.csv")
dummy_output_txt_path = os.path.join(OUTPUT_PATH, "dummy_chars.txt")
dummy_data = {'word': ['hello', 'world', 'python']}
pd.DataFrame(dummy_data).to_csv(dummy_csv_path, index=False)
print(f"Created a dummy CSV file at: {dummy_csv_path}")
convert_csv_to_chars_txt(dummy_csv_path, dummy_output_txt_path)


--- Demonstrating function with a dummy CSV file ---
Created a dummy CSV file at: /kaggle/working/dummy_data.csv
Successfully converted '/kaggle/working/dummy_data.csv' to '/kaggle/working/dummy_chars.txt', with one character per line.


In [7]:
print("Model files:")
!ls -larth {OUTPUT_PATH}/ckpt

Model files:
total 384M
drwxr-xr-x 3 root root 4.0K Jan  2 05:16 .cache
-rw-r--r-- 1 root root 4.6M Jan  2 05:16 content_encoder.safetensors
-rw-r--r-- 1 root root  79M Jan  2 05:16 style_encoder.safetensors
-rw-r--r-- 1 root root 301M Jan  2 05:16 unet.safetensors
drwxr-xr-x 3 root root 4.0K Jan  2 05:16 .
drwxr-xr-x 5 root root 4.0K Jan  2 05:16 ..


In [8]:
%cd {OUTPUT_PATH}
# ==========================================
# EXPORT / DOWNLOAD DATASET COMMANDS
# ==========================================
HF_USERNAME = "dzungpham"
# Train Split
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/train_original" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "train_original" \
  --token HF_TOKEN

!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/train" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "train" \
  --token HF_TOKEN
# Validation: Unseen Both
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/val" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "val" \
  --token HF_TOKEN

/kaggle/working

EXPORTING DATASET TO DISK

📥 Loading dataset from Hub...
   Repository: dzungpham/font-diffusion-generated-data
   Split: train_original
README.md: 3.05kB [00:00, 7.88MB/s]
train_original-00000-of-00001.parquet: 100%|██| 117M/117M [00:00<00:00, 208MB/s]
train-00000-of-00001.parquet: 100%|█████████| 77.1M/77.1M [00:00<00:00, 195MB/s]
val-00000-of-00001.parquet: 100%|███████████| 3.65M/3.65M [00:00<00:00, 120MB/s]
Generating train_original split: 100%|█| 8985/8985 [00:00<00:00, 29701.92 exampl
Generating val split: 100%|█████████| 357/357 [00:00<00:00, 35506.18 examples/s]
✓ Loaded dataset with 8985 samples from Hub

Exporting images from dataset...

🎨 Exporting images...
Exporting: 100%|███████████████████████████| 8985/8985 [00:21<00:00, 419.37it/s]
✓ Exported 599 content images
✓ Exported 8985 target images

💾 Saving results_checkpoint.json...
  ✓ Saved results_checkpoint.json (8985 generations)

📊 Metadata Statistics:
  Total generations: 8985
  Total characters: 599

In [9]:
print("Fonts currently in fonts/ folder")
!ls -lt FontDiffusion/fonts
print("Styles in style_images/ folder")
!ls -l FontDiffusion/styles_images

Fonts currently in fonts/ folder
total 332584
-rw-r--r-- 1 root root 26929728 Jan  2 05:14  NomNaTongLight2.ttf
-rw-r--r-- 1 root root 14574480 Jan  2 05:14  NomNaTongLight.ttf
-rw-r--r-- 1 root root 31729820 Jan  2 05:14  NomNaTong-Regular2.otf
-rw-r--r-- 1 root root 14574552 Jan  2 05:14  NomNaTong-Regular.ttf
-rw-r--r-- 1 root root  9424552 Jan  2 05:14  NomNaTong-Regular.otf
-rw-r--r-- 1 root root 12967288 Jan  2 05:14  HanaMinC.otf
-rw-r--r-- 1 root root 30739236 Jan  2 05:14  HanaMinB.ttf
-rw-r--r-- 1 root root 32201032 Jan  2 05:14  HanaMinB.otf
-rw-r--r-- 1 root root 22761228 Jan  2 05:14  HanaMinA.ttf
-rw-r--r-- 1 root root 31621108 Jan  2 05:14  HanaMinA.otf
-rw-r--r-- 1 root root 18202176 Jan  2 05:14 'Han-nom Minh 1.42.otf'
-rw-r--r-- 1 root root 19505228 Jan  2 05:14  Han-Nom-Khai-Regular-300623.ttf
-rw-r--r-- 1 root root 20368044 Jan  2 05:14 'Han-Nom Kai 1.00.otf'
-rw-r--r-- 1 root root 33815824 Jan  2 05:14 'HAN NOM B.ttf'
-rw-r--r-- 1 root root 21320444 Jan  2 05:14 'H

In [ ]:
if is_colab:
  !uv pip install --upgrade "huggingface-hub>=0.34.0,<1.0"
else:
  !uv pip install --upgrade "huggingface-hub==0.25.2" "protobuf<5.0.0" "numpy<2.0.0"
%cd {OUTPUT_PATH}
!accelerate launch --num_processes 1 \
    FontDiffusion/sample_batch.py \
    --characters "FontDiffusion/NomTuTao/Ds_10k_ChuNom_TuTao.txt" \
    --style_images "FontDiffusion/styles_images" \
    --ckpt_dir "ckpt/" \
    --ttf_path "FontDiffusion/fonts/NomNaTong-Regular.otf" \
    --output_dir "my_dataset/train_original" \
    --num_inference_steps 20 \
    --guidance_scale 7.5 \
    --start_line 301 \
    --end_line 600 \
    --batch_size 35 \
    --save_interval 1 \
    --channels_last \
    --seed 42 \
    --compile \
    --enable_xformers

In [ ]:
!find my_dataset/train_original/ContentImage -type f | wc -l
!find my_dataset/train_original/TargetImage -type f | wc -l

In [ ]:
# !ls -lt my_dataset/train_original/ContentImage/*
# !ls -l my_dataset/train_original/TargetImage/*hanhthu*
# !ls -lt my_dataset/train_original/TargetImage/*

In [ ]:
import re
from pathlib import Path

# Your valid pattern
expected_pattern = r"U\+[0-9A-F]{4,5}.*_[0-9a-f]{8}\.png"

# Define the root directory ('.' for current directory)
root_dir = Path('./my_dataset/train_original/TargetImage')

# .rglob('*') finds every file recursively
for path in root_dir.rglob('*'):
    # Process only files (ignore directories)
    if path.is_file():
        # Check if the FILENAME (path.name) matches the regex
        if not re.match(expected_pattern, path.name):
            try:
                print(f"Deleting invalid file: {path}")
                # path.unlink() # This deletes the file
            except Exception as e:
                print(f"Error deleting {path}: {e}")

In [ ]:
!python FontDiffusion/create_validation_split.py \
  --data_root my_dataset \
  --val_ratio 0.2 \
  --seed 42

In [ ]:
!uv pip install --upgrade pyarrow datasets

In [ ]:
# remove_unparseable_files.py

with open("my_dataset/unparseable_files.txt", "r", encoding="utf-8") as f:
    paths = [line.strip() for line in f if line.strip()]

import os

for path in paths:
    try:
        if os.path.exists(path):
            os.remove(path)
            print(f"Deleted: {path}")
        else:
            print(f"Not found: {path}")
    except Exception as e:
        print(f"Error deleting {path}: {e}")

In [ ]:
HF_USERNAME = "dzungpham"
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/train_original" \
  --repo_id dzungpham/font-diffusion-generated-data \
  --split "train_original" \
  --token {HF_TOKEN}

# Train Split
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/train" \
  --repo_id dzungpham/font-diffusion-generated-data \
  --split "train" \
  --token {HF_TOKEN}

# Train Split
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/val" \
  --repo_id dzungpham/font-diffusion-generated-data \
  --split "val" \
  --token {HF_TOKEN}


In [ ]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

In [21]:
# TRAINING PHASE 1
if is_colab:
  !uv pip install --upgrade "huggingface-hub>=0.34.0,<1.0"
else:
  !uv pip install --upgrade "huggingface-hub==0.25.2"
import wandb

MAX_TRAIN_STEPS = 500
!accelerate launch FontDiffusion/my_train.py \
    --seed=123 \
    --experience_name="FontDiffuser_training_phase_1" \
    --data_root="my_dataset" \
    --phase_1_ckpt_dir="outputs/FontDiffuser/global_step_800" \
    --output_dir="outputs/FontDiffuser" \
    --report_to="wandb" \
    \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
    \
    --train_batch_size=8 \
    --gradient_accumulation_steps=2 \
    --perceptual_coefficient=0.05 \
    --offset_coefficient=0.7 \
    --max_train_steps={MAX_TRAIN_STEPS} \
    --ckpt_interval={MAX_TRAIN_STEPS // 2} \
    --log_interval=50 \
    \
    --learning_rate=1e-4 \
    --lr_scheduler="linear" \
    --lr_warmup_steps=10000 \
    --drop_prob=0.1 \
    --mixed_precision="no"

import logging
logging.info("Finetuned model checkpoints:")
!ls -lr outputs/FontDiffuser

Using Python 3.11.13 environment at: /usr
Resolved 12 packages in 90ms                                         
Audited 12 packages in 0.15ms
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydan

In [22]:
!ls -lt ckpt

content_encoder.safetensors  style_encoder.safetensors	unet.safetensors


In [ ]:
# TRAINING PHASE 2
!wandb login
!python FontDiffusion/my_train.py \
    --seed=123 \
    --experience_name="FontDiffuser_training_phase_2" \
    --data_root="my_dataset" \
    --output_dir="outputs/FontDiffuser" \
    --report_to="wandb" \
    --phase_2 \
    --phase_1_ckpt_dir="outputs/FontDiffuser/global_step_800" \
    --scr_ckpt_path="ckpt/scr_210000.pth" \
    --sc_coefficient=0.05 \
    --num_neg=13 \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
    --train_batch_size=8 \
    --perceptual_coefficient=0.03 \
    --offset_coefficient=0.4 \
    --max_train_steps=100 \
    --ckpt_interval=50 \
    --gradient_accumulation_steps=2 \
    --log_interval=50 \
    --learning_rate=1e-5 \
    --lr_scheduler="constant" \
    --lr_warmup_steps=1000 \
    --drop_prob=0.1 \
    --mixed_precision="no"


In [25]:
!ls -l outputs/FontDiffuser/global_step_800

total 785332
-rw-r--r-- 1 root root   4756580 Jan  2 06:04 content_encoder.safetensors
-rw-r--r-- 1 root root  82394556 Jan  2 06:04 style_encoder.safetensors
-rw-r--r-- 1 root root 402084924 Jan  2 06:04 total_model.safetensors
-rw-r--r-- 1 root root 314927748 Jan  2 06:04 unet.safetensors


In [26]:
!python FontDiffusion/pth2safetensors.py \
    --weights_dir "outputs/FontDiffuser/global_step_800" \
    --repo_id "dzungpham/font-diffusion-weights" \
    --token "{HF_TOKEN}"


PYTORCH TO SAFETENSORS CONVERTER & HF UPLOADER

VALIDATING INPUTS
✓ Weights directory: outputs/FontDiffuser/global_step_800
  Contents: 4 files
✓ Repository ID: dzungpham/font-diffusion-weights
✓ HF token: ********************

✓ Files to process: 5
  ⚠ content_encoder.pth (not found)
  ⚠ style_encoder.pth (not found)
  ⚠ unet.pth (not found)
  ⚠ total_model.pth (not found)
  ⚠ scr.pth (not found)

CONVERTING .pth TO SAFETENSORS

⚠ content_encoder.pth: Not found, skipping

⚠ style_encoder.pth: Not found, skipping

⚠ unet.pth: Not found, skipping

⚠ total_model.pth: Not found, skipping

⚠ scr.pth: Not found, skipping

----------------------------------------------------------------------
Conversion complete: 0 succeeded, 0 failed

UPLOADING TO HUGGING FACE HUB
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
To

In [ ]:
import os
import zipfile
from pathlib import Path
from typing import List
def find_result_folders(base_path: Path, pattern_name: str) -> List[Path]:
    return [p for p in base_path.glob(pattern_name) if p.is_dir()]

def zip_folder(folder_path: Path, output_base_path: Path) -> bool:
    folder_name = folder_path.name
    zip_path = output_base_path / f"{folder_name}.zip"
    try:
        print(f"   -> Zipping folder: {folder_name}...")
        with zipfile.ZipFile(zip_path, mode="w", compression=zipfile.ZIP_DEFLATED) as zipf:
            for file_path in folder_path.rglob("*"):
                if file_path.is_file():
                    arcname = file_path.relative_to(folder_path.parent)
                    zipf.write(file_path, arcname)
        print(f"   ✅ Created ZIP: {zip_path.name}")
        return True
    except Exception as exc:
        print(f"   ❌ Failed to zip {folder_name}: {exc}")
        return False

def zip_stats_results_folders(output_base_path: str, pattern_name: str) -> None:
    base = Path(output_base_path)
    base.mkdir(parents=True, exist_ok=True)
    result_folders = find_result_folders(base, pattern_name)
    if not result_folders:
        print(f"⚠️ No folders matching '*dataset' found in '{output_base_path}'.")
        return
    print(f"🔍 Found {len(result_folders)} result folder(s) to zip.")
    successful = sum(1 for folder in result_folders if zip_folder(folder, base))
    print(f"\n✅ DONE! Successfully zipped {successful} out of {len(result_folders)} folder(s).")

if __name__ == "__main__":
    try:
        output_root = os.getenv("OUTPUT_PATH") or globals().get("OUTPUT_PATH")
        if not output_root:
            raise ValueError("OUTPUT_PATH not defined")
        zip_stats_results_folders(
            output_base_path=OUTPUT_PATH,
            pattern_name="my_dataset")
    except Exception as e:
        print(f"❌ An error occurred: {e}")